In [1]:
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import SubsetRandomSampler, ConcatDataset
from torch.utils.data import random_split

import torchvision
from torchvision.io import read_image
from torchvision import transforms

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [13]:
class CNN(nn.Module):
    def __init__(self, input_d, output_d, layer_s, dropout):
        super(CNN, self).__init__()
        self.myn = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(input_d,layer_s[0]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(layer_s[0],layer_s[1]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(layer_s[1], layer_s[2]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(layer_s[2], layer_s[3]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(layer_s[3], layer_s[4]),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(layer_s[4], output_d),
        )
        
    def forward(self, input_tensor):
        output_tensor = self.myn(input_tensor)
        return output_tensor

In [19]:
model = CNN(x_train_tensor.shape[1], len(torch.unique(y_train_tensor)), [1024,1024,512,256,128], 0.2)

In [20]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
epoches = 1000
loss_vals_train = []

In [24]:
for i in range(epoches):
    y_pred = model(x_train_tensor)
    loss = loss_func(y_pred[:-15], y_train_tensor[:-15])
    loss_vals_train.append(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [25]:
model.eval()

CNN(
  (myn): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=6, out_features=1024, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=1024, out_features=512, bias=True)
    (8): ReLU()
    (9): Dropout(p=0.2, inplace=False)
    (10): Linear(in_features=512, out_features=256, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.2, inplace=False)
    (13): Linear(in_features=256, out_features=128, bias=True)
    (14): ReLU()
    (15): Dropout(p=0.2, inplace=False)
    (16): Linear(in_features=128, out_features=6, bias=True)
  )
)

In [26]:
y_pred = model(x_test_tensor)
y_pred = np.argmax(y_pred.detach().numpy(), axis = 1)
accuracy_score(y_test_tensor, y_pred)

1.0